In [58]:
import requests


item_url = "https://penguin-stats.io/PenguinStats/api/v2/items"
matrix_url = "https://penguin-stats.io/PenguinStats/api/v2/result/matrix?show_closed_zones=true"
stage_url = "https://penguin-stats.io/PenguinStats/api/v2/stages"
zone_url = "https://penguin-stats.io/PenguinStats/api/v2/zones"

item_data = requests.get(item_url).json()
matrix_data = requests.get(matrix_url).json()
stage_data = requests.get(stage_url).json()
zone_data = requests.get(zone_url).json()


In [59]:
item_dict = {item_info["itemId"]: item_info for item_info in item_data}
stage_dict = {stage_info["stageId"]: stage_info for stage_info in stage_data}
zone_dict = {zone_info["zoneId"]: zone_info for zone_info in zone_data}


In [60]:
from datetime import datetime, timezone, timedelta

def get_item_name(item_id):
    return item_dict[item_id]["name"]

def get_stage_name(stage_id):
    return stage_dict[stage_id]["code"]

def get_stage_open_timestamp(stage_id, server):
    return stage_dict[stage_id]["existence"][server]["openTime"]

TIMEZONE_OFFSET_HOURS_DICT = {
    "CN": +8,
    "US": -5,
    "JP": +9,
    "KR": +9,
}
def get_timezone(server):
    return timezone(timedelta(hours=TIMEZONE_OFFSET_HOURS_DICT[server]))

def get_stage_open_time(stage_id, server):
    return datetime.fromtimestamp(get_stage_open_timestamp(stage_id, server) / 1000).astimezone(get_timezone(server))

def get_stage_ap_cost(stage_id):
    return stage_dict[stage_id]["apCost"]

def get_zone_name(zone_id):
    return zone_dict[zone_id]["zoneName"]

def get_zone_name_by_stage_id(stage_id):
    return get_zone_name(stage_dict[stage_id]["zoneId"])


In [61]:
SERVER = "CN"
TIMES_THRESHOLD = 256

stages = []
for stage_id, stage_info in stage_dict.items():
    if stage_info["stageType"] == "ACTIVITY" and stage_info["zoneId"] != "gachabox":
        stages.append(stage_info["stageId"])

stage_drop_info = {stage_id: {} for stage_id in stages}
for element in matrix_data["matrix"]:
    stage_id = element["stageId"]
    if stage_id not in stage_drop_info:
        continue

    item_id = element["itemId"]
    times = element["times"]
    quantity = element["quantity"]
    assert item_id not in stage_drop_info[stage_id]
    if times >= TIMES_THRESHOLD and item_id != "furni":
        stage_drop_info[stage_id][item_id] = (quantity, times)

# 按掉落频率排序
stage_drop_info = {k: sorted(v.items(), key=lambda x: x[1][0] / x[1][1], reverse=True) for k, v in stage_drop_info.items() if v}
# 按作战时间排序
stage_drop_info = sorted(stage_drop_info.items(), key=lambda x: get_stage_open_timestamp(x[0], SERVER))


作战名称	活动名称	作战开放时间	作战理智消耗	作战掉落物品名称	掉落数	样本数	作战掉落物品数量	单位理智掉落物品数量
GT-5	骑兵与猎人	2019-05-30 10:00:00+08:00	15	扭转醇	8073	12211	0.6611252150	0.0440750143
GT-6	骑兵与猎人	2019-05-30 10:00:00+08:00	15	研磨石	5208	10649	0.4890600056	0.0326040004
OF-7	火蓝之心	2019-08-27 16:00:00+08:00	20	汐斯塔的黑曜石	14560	364	40.0000000000	2.0000000000
OF-8	火蓝之心	2019-08-27 16:00:00+08:00	20	黑曜石节门票	4180	418	10.0000000000	0.5000000000
OF-F3	火蓝之心	2019-08-27 16:00:00+08:00	30	黑曜石节抽奖代币	22448	702	31.9772079772	1.0659069326
OF-F4	火蓝之心	2019-08-27 16:00:00+08:00	40	黑曜石节抽奖代币	79865	999	79.9449449449	1.9986236236
DM-6	生于黑夜	2020-04-21 16:00:00+08:00	15	糖组	7947	11459	0.6935160136	0.0462344009
DM-7	生于黑夜	2020-04-21 16:00:00+08:00	18	扭转醇	18756	23847	0.7865140269	0.0436952237
DM-8	生于黑夜	2020-04-21 16:00:00+08:00	18	异铁组	29777	44625	0.6672717087	0.0370706505
TW-6	沃伦姆德的薄暮	2020-07-09 16:00:00+08:00	15	炽合金	1441	2436	0.5915435140	0.0394362343
TW-7	沃伦姆德的薄暮	2020-07-09 16:00:00+08:00	18	酮凝集组	18766	27842	0.6740176711	0.0374454262
TW-8	沃伦姆德的薄暮	2020-07-09 16:00:0

In [ ]:
print("作战名称\t活动名称\t作战开放时间\t作战理智消耗\t作战掉落物品名称\t掉落数\t样本数\t作战掉落物品数量\t单位理智掉落物品数量")
for stage_id, drop_info in stage_drop_info:
    if len(drop_info) == 1:
        stage_info = stage_dict[stage_id]
        stage_name = get_stage_name(stage_id)
        zone_name = get_zone_name_by_stage_id(stage_id)
        stage_open_time = get_stage_open_time(stage_id, SERVER)
        ap_cost = get_stage_ap_cost(stage_id)
        print(f"{stage_name}\t{zone_name}\t{stage_open_time}\t{ap_cost}\t{"\t".join(f"{get_item_name(item_id)}\t{quantity}\t{times}\t{quantity / times:.10f}\t{quantity / times / ap_cost:.10f}" for item_id, (quantity, times) in drop_info)}")


In [71]:
import pandas as pd

columns = ['作战名称', '活动名称', '作战开放时间', '作战理智消耗', '作战掉落物品名称', '掉落数', '样本数']
records = []

for stage_id, drop_info in stage_drop_info:
    stage_name = get_stage_name(stage_id)
    zone_name = get_zone_name_by_stage_id(stage_id)
    stage_open_time = get_stage_open_time(stage_id, SERVER)
    ap_cost = get_stage_ap_cost(stage_id)
    if len(drop_info) == 1:
        item_id, (quantity, times) = drop_info[0]
        item_name = get_item_name(item_id)
        records.append({
            '作战名称': stage_name,
            '活动名称': zone_name,
            '作战开放时间': stage_open_time,
            '作战理智消耗': ap_cost,
            '作战掉落物品名称': item_name,
            '掉落数': quantity,
            '样本数': times,
        })

df = pd.DataFrame.from_records(records, columns=columns)
df


,作战名称,活动名称,作战开放时间,作战理智消耗,作战掉落物品名称,掉落数,样本数
0,GT-5,骑兵与猎人,2019-05-30 10:00:00+08:00,15,扭转醇,8073,12211
1,GT-6,骑兵与猎人,2019-05-30 10:00:00+08:00,15,研磨石,5208,10649
2,OF-7,火蓝之心,2019-08-27 16:00:00+08:00,20,汐斯塔的黑曜石,14560,364
3,OF-8,火蓝之心,2019-08-27 16:00:00+08:00,20,黑曜石节门票,4180,418
4,OF-F3,火蓝之心,2019-08-27 16:00:00+08:00,30,黑曜石节抽奖代币,22448,702
...,...,...,...,...,...,...,...
169,CV-7,不义之财・复刻,2024-09-19 16:00:00+08:00,21,研磨石,677328,1000203
170,CV-8,不义之财・复刻,2024-09-19 16:00:00+08:00,21,转质盐组,228570,366243
171,GO-7,追迹日落以西,2024-10-09 16:00:00+08:00,21,固源岩组,125696,114776
172,GO-8,追迹日落以西,2024-10-09 16:00:00+08:00,21,凝胶,44478,63218


In [62]:
raw_data = """作战名称	活动名称	作战开放时间	作战理智消耗	作战掉落物品名称	掉落数	样本数	作战掉落物品数量	单位理智掉落物品数量
GT-5	骑兵与猎人	2019-05-30 10:00:00+08:00	15	扭转醇	8073	12211	0.6611252150	0.0440750143
GT-6	骑兵与猎人	2019-05-30 10:00:00+08:00	15	研磨石	5208	10649	0.4890600056	0.0326040004
OF-7	火蓝之心	2019-08-27 16:00:00+08:00	20	汐斯塔的黑曜石	14560	364	40.0000000000	2.0000000000
OF-8	火蓝之心	2019-08-27 16:00:00+08:00	20	黑曜石节门票	4180	418	10.0000000000	0.5000000000
OF-F3	火蓝之心	2019-08-27 16:00:00+08:00	30	黑曜石节抽奖代币	22448	702	31.9772079772	1.0659069326
OF-F4	火蓝之心	2019-08-27 16:00:00+08:00	40	黑曜石节抽奖代币	79865	999	79.9449449449	1.9986236236
DM-6	生于黑夜	2020-04-21 16:00:00+08:00	15	糖组	7947	11459	0.6935160136	0.0462344009
DM-7	生于黑夜	2020-04-21 16:00:00+08:00	18	扭转醇	18756	23847	0.7865140269	0.0436952237
DM-8	生于黑夜	2020-04-21 16:00:00+08:00	18	异铁组	29777	44625	0.6672717087	0.0370706505
TW-6	沃伦姆德的薄暮	2020-07-09 16:00:00+08:00	15	炽合金	1441	2436	0.5915435140	0.0394362343
TW-7	沃伦姆德的薄暮	2020-07-09 16:00:00+08:00	18	酮凝集组	18766	27842	0.6740176711	0.0374454262
TW-8	沃伦姆德的薄暮	2020-07-09 16:00:00+08:00	18	研磨石	29055	50174	0.5790847849	0.0321713769
RI-6	密林悍将归来	2020-08-25 16:00:00+08:00	15	固源岩组	5103	6586	0.7748253872	0.0516550258
RI-7	密林悍将归来	2020-08-25 16:00:00+08:00	18	糖组	2412	2908	0.8294360385	0.0460797799
RI-8	密林悍将归来	2020-08-25 16:00:00+08:00	18	RMA70-12	24140	48029	0.5026130046	0.0279229447
GT-5	骑兵与猎人・复刻	2020-10-01 16:00:00+08:00	15	扭转醇	19547	29749	0.6570641030	0.0438042735
GT-6	骑兵与猎人・复刻	2020-10-01 16:00:00+08:00	15	研磨石	6755	14044	0.4809883224	0.0320658882
MN-6	玛莉娅・临光	2020-10-15 16:00:00+08:00	15	凝胶	750	1515	0.4950495050	0.0330033003
MN-7	玛莉娅・临光	2020-10-15 16:00:00+08:00	18	聚酸酯组	4670	5601	0.8337796822	0.0463210935
MN-8	玛莉娅・临光	2020-10-15 16:00:00+08:00	18	轻锰矿	22363	33204	0.6735031924	0.0374168440
MB-6	孤岛风云	2020-12-17 16:00:00+08:00	15	糖组	5253	7494	0.7009607686	0.0467307179
MB-7	孤岛风云	2020-12-17 16:00:00+08:00	18	酮凝集组	16503	24552	0.6721652004	0.0373425111
MB-8	孤岛风云	2020-12-17 16:00:00+08:00	18	异铁组	9774	14532	0.6725846408	0.0373658134
WR-8	画中人	2021-02-05 16:00:00+08:00	15	炽合金	24693	41459	0.5956004728	0.0397066982
WR-9	画中人	2021-02-05 16:00:00+08:00	18	聚酸酯组	5383	6513	0.8265008445	0.0459167136
WR-10	画中人	2021-02-05 16:00:00+08:00	18	固源岩组	11996	12881	0.9312941542	0.0517385641
OD-7	源石尘行动	2021-03-09 16:00:00+08:00	18	轻锰矿	13661	20382	0.6702482583	0.0372360143
OD-8	源石尘行动	2021-03-09 16:00:00+08:00	18	研磨石	16596	28546	0.5813774259	0.0322987459
DM-6	生于黑夜・复刻	2021-03-30 16:00:00+08:00	15	糖组	3570	5093	0.7009621048	0.0467308070
DM-7	生于黑夜・复刻	2021-03-30 16:00:00+08:00	18	扭转醇	11981	15211	0.7876536717	0.0437585373
DM-8	生于黑夜・复刻	2021-03-30 16:00:00+08:00	18	异铁组	7990	12060	0.6625207297	0.0368067072
WD-6	遗尘漫步	2021-04-15 16:00:00+08:00	15	聚酸酯组	3921	5723	0.6851301765	0.0456753451
WD-7	遗尘漫步	2021-04-15 16:00:00+08:00	18	酮凝集组	8710	13142	0.6627606148	0.0368200342
WD-8	遗尘漫步	2021-04-15 16:00:00+08:00	18	RMA70-12	23176	46692	0.4963591193	0.0275755066
SV-7	覆潮之下	2021-05-01 16:00:00+08:00	15	固源岩组	5446	6980	0.7802292264	0.0520152818
SV-8	覆潮之下	2021-05-01 16:00:00+08:00	18	晶体元件	7902	10681	0.7398183691	0.0411010205
SV-9	覆潮之下	2021-05-01 16:00:00+08:00	18	全新装置	58144	116534	0.4989445140	0.0277191397
TW-6	沃伦姆德的薄暮・复刻	2021-06-17 16:00:00+08:00	15	炽合金	16233	27047	0.6001774689	0.0400118313
TW-7	沃伦姆德的薄暮・复刻	2021-06-17 16:00:00+08:00	18	酮凝集组	6407	9507	0.6739244767	0.0374402487
TW-8	沃伦姆德的薄暮・复刻	2021-06-17 16:00:00+08:00	18	研磨石	14872	25806	0.5763000853	0.0320166714
RI-6	密林悍将归来・复刻	2021-07-20 16:00:00+08:00	15	固源岩组	7450	9482	0.7856992196	0.0523799480
RI-7	密林悍将归来・复刻	2021-07-20 16:00:00+08:00	18	糖组	5400	6541	0.8255618407	0.0458645467
RI-8	密林悍将归来・复刻	2021-07-20 16:00:00+08:00	18	RMA70-12	13356	27251	0.4901104547	0.0272283586
MN-6	玛莉娅・临光・复刻	2021-10-01 16:00:00+08:00	15	凝胶	17606	35223	0.4998438520	0.0333229235
MN-7	玛莉娅・临光・复刻	2021-10-01 16:00:00+08:00	18	聚酸酯组	2405	2916	0.8247599451	0.0458199970
MN-8	玛莉娅・临光・复刻	2021-10-01 16:00:00+08:00	18	轻锰矿	2885	4315	0.6685979143	0.0371443286
NL-8	长夜临光	2021-11-01 16:00:00+08:00	18	糖组	6424	7844	0.8189699133	0.0454983285
NL-9	长夜临光	2021-11-01 16:00:00+08:00	18	晶体元件	10845	14569	0.7443887707	0.0413549317
NL-10	长夜临光	2021-11-01 16:00:00+08:00	18	扭转醇	24614	31473	0.7820671687	0.0434481760
MB-6	孤岛风云・复刻	2021-12-07 16:00:00+08:00	15	糖组	2269	3311	0.6852914527	0.0456860968
MB-7	孤岛风云・复刻	2021-12-07 16:00:00+08:00	18	酮凝集组	9650	14489	0.6660224998	0.0370012500
MB-8	孤岛风云・复刻	2021-12-07 16:00:00+08:00	18	异铁组	18765	28641	0.6551796376	0.0363988688
BI-6	风雪过境	2021-12-21 16:00:00+08:00	15	炽合金	12787	22017	0.5807784893	0.0387185660
BI-7	风雪过境	2021-12-21 16:00:00+08:00	18	聚酸酯组	5091	6057	0.8405151065	0.0466952837
BI-8	风雪过境	2021-12-21 16:00:00+08:00	18	研磨石	36646	64657	0.5667754458	0.0314875248
WR-8	画中人・复刻	2022-01-10 16:00:00+08:00	15	炽合金	19288	32914	0.5860120314	0.0390674688
WR-9	画中人・复刻	2022-01-10 16:00:00+08:00	18	聚酸酯组	2915	3520	0.8281250000	0.0460069444
WR-10	画中人・复刻	2022-01-10 16:00:00+08:00	18	固源岩组	19802	21146	0.9364418803	0.0520245489
IW-6	将进酒	2022-01-25 16:00:00+08:00	18	全新装置	62273	125383	0.4966622269	0.0275923459
IW-7	将进酒	2022-01-25 16:00:00+08:00	21	固源岩组	31016	28216	1.0992344769	0.0523444989
IW-8	将进酒	2022-01-25 16:00:00+08:00	21	RMA70-12	34302	59181	0.5796116997	0.0276005571
GA-6	吾导先路	2022-03-15 16:00:00+08:00	18	糖组	30079	36028	0.8348784279	0.0463821349
GA-7	吾导先路	2022-03-15 16:00:00+08:00	21	扭转醇	48388	52561	0.9206065334	0.0438384064
GA-8	吾导先路	2022-03-15 16:00:00+08:00	21	凝胶	96904	138864	0.6978338518	0.0332301834
WD-6	遗尘漫步・复刻	2022-03-29 16:00:00+08:00	15	聚酸酯组	8494	12217	0.6952607023	0.0463507135
WD-7	遗尘漫步・复刻	2022-03-29 16:00:00+08:00	18	酮凝集组	40931	61195	0.6688618351	0.0371589908
WD-8	遗尘漫步・复刻	2022-03-29 16:00:00+08:00	18	RMA70-12	23995	48105	0.4988046981	0.0277113721
SN-8	愚人号	2022-05-01 16:00:00+08:00	21	异铁组	249007	320632	0.7766130642	0.0369815745
SN-9	愚人号	2022-05-01 16:00:00+08:00	21	轻锰矿	293869	377968	0.7774970368	0.0370236684
SN-10	愚人号	2022-05-01 16:00:00+08:00	21	化合切削液	105855	151203	0.7000853158	0.0333373960
SV-7	覆潮之下・复刻	2022-05-30 16:00:00+08:00	15	固源岩组	114679	146895	0.7806868852	0.0520457923
SV-8	覆潮之下・复刻	2022-05-30 16:00:00+08:00	18	晶体元件	15630	20849	0.7496762435	0.0416486802
SV-9	覆潮之下・复刻	2022-05-30 16:00:00+08:00	18	全新装置	247699	496180	0.4992119795	0.0277339989
LE-5	尘影余音	2022-06-09 16:00:00+08:00	18	糖组	88277	105968	0.8330533746	0.0462807430
LE-6	尘影余音	2022-06-09 16:00:00+08:00	21	研磨石	346113	511895	0.6761406148	0.0321971721
LE-7	尘影余音	2022-06-09 16:00:00+08:00	21	半自然溶剂	89588	128165	0.6990051886	0.0332859614
DV-6	绿野幻梦	2022-07-05 16:00:00+08:00	21	聚酸酯组	210053	216490	0.9702665250	0.0462031679
DV-7	绿野幻梦	2022-07-05 16:00:00+08:00	21	扭转醇	364290	395999	0.9199265655	0.0438060269
DV-8	绿野幻梦	2022-07-05 16:00:00+08:00	21	炽合金	584809	702487	0.8324837328	0.0396420825
IC-7	理想城：长夏狂欢季	2022-08-11 16:00:00+08:00	21	酮凝集组	731012	942905	0.7752764064	0.0369179241
IC-8	理想城：长夏狂欢季	2022-08-11 16:00:00+08:00	21	凝胶	265540	380237	0.6983539214	0.0332549486
IC-9	理想城：长夏狂欢季	2022-08-11 16:00:00+08:00	21	RMA70-12	951728	1642921	0.5792901789	0.0275852466
NL-8	长夜临光・复刻	2022-09-22 16:00:00+08:00	18	糖组	374072	449537	0.8321272776	0.0462292932
NL-9	长夜临光・复刻	2022-09-22 16:00:00+08:00	18	晶体元件	106156	141871	0.7482572196	0.0415698455
NL-10	长夜临光・复刻	2022-09-22 16:00:00+08:00	18	扭转醇	450390	571018	0.7887492163	0.0438194009
IS-8	叙拉古人	2022-11-01 16:00:00+08:00	21	固源岩组	4465516	4081599	1.0940604405	0.0520981162
IS-9	叙拉古人	2022-11-01 16:00:00+08:00	21	化合切削液	325934	465733	0.6998301602	0.0333252457
IS-10	叙拉古人	2022-11-01 16:00:00+08:00	21	炽合金	715335	859664	0.8321099872	0.0396242851
BI-6	风雪过境・复刻	2022-12-01 16:00:00+08:00	15	炽合金	27419	46165	0.5939347991	0.0395956533
BI-7	风雪过境・复刻	2022-12-01 16:00:00+08:00	18	聚酸酯组	86863	104598	0.8304460888	0.0461358938
BI-8	风雪过境・复刻	2022-12-01 16:00:00+08:00	18	研磨石	176917	306620	0.5769910639	0.0320550591
FC-5	照我以火	2022-12-15 16:00:00+08:00	21	糖组	517862	533053	0.9715018957	0.0462619950
FC-6	照我以火	2022-12-15 16:00:00+08:00	21	异铁组	3263055	4193787	0.7780688433	0.0370508973
FC-7	照我以火	2022-12-15 16:00:00+08:00	21	半自然溶剂	298824	427024	0.6997826820	0.0333229849
IW-6	将进酒・复刻	2023-01-01 16:00:00+08:00	18	全新装置	1203636	2404446	0.5005876614	0.0278104256
IW-7	将进酒・复刻	2023-01-01 16:00:00+08:00	21	固源岩组	750116	685428	1.0943760687	0.0521131461
IW-8	将进酒・复刻	2023-01-01 16:00:00+08:00	21	RMA70-12	763250	1309669	0.5827808400	0.0277514686
WB-7	登临意	2023-01-17 16:00:00+08:00	21	酮凝集组	1773946	2281344	0.7775881235	0.0370280059
WB-8	登临意	2023-01-17 16:00:00+08:00	21	轻锰矿	3477457	4469136	0.7781049849	0.0370526183
WB-9	登临意	2023-01-17 16:00:00+08:00	21	晶体元件	306185	350200	0.8743146773	0.0416340323
GA-6	吾导先路・复刻	2023-02-21 16:00:00+08:00	18	糖组	648278	778372	0.8328639776	0.0462702210
GA-7	吾导先路・复刻	2023-02-21 16:00:00+08:00	21	扭转醇	2263352	2459805	0.9201347261	0.0438159393
GA-8	吾导先路・复刻	2023-02-21 16:00:00+08:00	21	凝胶	1609884	2292099	0.7023623325	0.0334458254
SN-8	愚人号・复刻	2023-04-20 16:00:00+08:00	21	异铁组	2560483	3291292	0.7779568024	0.0370455620
SN-9	愚人号・复刻	2023-04-20 16:00:00+08:00	21	轻锰矿	2122944	2728969	0.7779289541	0.0370442359
SN-10	愚人号・复刻	2023-04-20 16:00:00+08:00	21	化合切削液	477884	682207	0.7004970632	0.0333570030
CW-8	孤星	2023-05-01 16:00:00+08:00	21	酮凝集组	4640251	5961652	0.7783498601	0.0370642791
CW-9	孤星	2023-05-01 16:00:00+08:00	21	半自然溶剂	603188	861878	0.6998531115	0.0333263386
CW-10	孤星	2023-05-01 16:00:00+08:00	21	炽合金	7135136	8567650	0.8327996592	0.0396571266
HE-6	空想花庭	2023-06-08 16:00:00+08:00	21	固源岩组	4888777	4468116	1.0941472871	0.0521022518
HE-7	空想花庭	2023-06-08 16:00:00+08:00	21	凝胶	2380543	3390658	0.7020887981	0.0334327999
HE-8	空想花庭	2023-06-08 16:00:00+08:00	21	RMA70-12	2228123	3825067	0.5825056136	0.0277383626
LE-5	尘影余音・复刻	2023-06-20 16:00:00+08:00	18	糖组	859425	1032060	0.8327277484	0.0462626527
LE-6	尘影余音・复刻	2023-06-20 16:00:00+08:00	21	研磨石	3791790	5603884	0.6766360617	0.0322207648
LE-7	尘影余音・复刻	2023-06-20 16:00:00+08:00	21	半自然溶剂	280541	401552	0.6986417699	0.0332686557
DV-6	绿野幻梦・复刻	2023-07-18 16:00:00+08:00	21	聚酸酯组	1691424	1740127	0.9720118129	0.0462862768
DV-7	绿野幻梦・复刻	2023-07-18 16:00:00+08:00	21	扭转醇	3246658	3526208	0.9207222036	0.0438439145
DV-8	绿野幻梦・复刻	2023-07-18 16:00:00+08:00	21	炽合金	789927	948557	0.8327670346	0.0396555731
SL-6	火山旅梦	2023-08-01 16:00:00+08:00	21	酮凝集组	5725563	7355679	0.7783867404	0.0370660353
SL-7	火山旅梦	2023-08-01 16:00:00+08:00	21	晶体元件	5354272	6120566	0.8748001410	0.0416571496
SL-8	火山旅梦	2023-08-01 16:00:00+08:00	21	轻锰矿	7971823	10241794	0.7783619745	0.0370648559
IC-7	理想城：长夏狂欢季・复刻	2023-08-31 16:00:00+08:00	21	酮凝集组	246819	317058	0.7784664005	0.0370698286
IC-8	理想城：长夏狂欢季・复刻	2023-08-31 16:00:00+08:00	21	凝胶	1623858	2312296	0.7022708165	0.0334414675
IC-9	理想城：长夏狂欢季・复刻	2023-08-31 16:00:00+08:00	21	RMA70-12	1903985	3266116	0.5829508199	0.0277595629
CV-6	不义之财	2023-09-05 16:00:00+08:00	21	糖组	2076512	2137030	0.9716812586	0.0462705361
CV-7	不义之财	2023-09-05 16:00:00+08:00	21	研磨石	3128673	4623640	0.6766688150	0.0322223245
CV-8	不义之财	2023-09-05 16:00:00+08:00	21	转质盐组	837905	1341614	0.6245499823	0.0297404753
IS-8	叙拉古人・复刻	2023-10-22 16:00:00+08:00	21	固源岩组	5411533	4948023	1.0936757974	0.0520797999
IS-9	叙拉古人・复刻	2023-10-22 16:00:00+08:00	21	化合切削液	461316	658598	0.7004515653	0.0333548364
IS-10	叙拉古人・复刻	2023-10-22 16:00:00+08:00	21	炽合金	1571150	1887297	0.8324868847	0.0396422326
RS-6	银心湖列车	2023-12-05 16:00:00+08:00	21	聚酸酯组	4078808	4196827	0.9718789933	0.0462799521
RS-7	银心湖列车	2023-12-05 16:00:00+08:00	21	轻锰矿	4242907	5452314	0.7781846387	0.0370564114
RS-8	银心湖列车	2023-12-05 16:00:00+08:00	21	晶体元件	1250044	1427324	0.8757955447	0.0417045497
FC-5	照我以火・复刻	2023-12-22 16:00:00+08:00	21	糖组	1577424	1622856	0.9720049099	0.0462859481
FC-6	照我以火・复刻	2023-12-22 16:00:00+08:00	21	异铁组	3563282	4577287	0.7784703035	0.0370700145
FC-7	照我以火・复刻	2023-12-22 16:00:00+08:00	21	半自然溶剂	388301	555158	0.6994423209	0.0333067772
WB-7	登临意・复刻	2024-01-18 16:00:00+08:00	21	酮凝集组	3067556	3939881	0.7785910285	0.0370757633
WB-8	登临意・复刻	2024-01-18 16:00:00+08:00	21	轻锰矿	1550794	1992479	0.7783238870	0.0370630422
WB-9	登临意・复刻	2024-01-18 16:00:00+08:00	21	晶体元件	1119571	1280716	0.8741758516	0.0416274215
HS-7	怀黍离	2024-02-01 16:00:00+08:00	21	固源岩组	5385338	4923757	1.0937456905	0.0520831281
HS-8	怀黍离	2024-02-01 16:00:00+08:00	21	扭转醇	4001041	4344298	0.9209867739	0.0438565130
HS-9	怀黍离	2024-02-01 16:00:00+08:00	21	RMA70-12	6892613	11828009	0.5827365366	0.0277493589
CR-6	水晶箭行动	2024-03-07 16:00:00+08:00	21	聚酸酯组	690623	710872	0.9715152658	0.0462626317
CR-7	水晶箭行动	2024-03-07 16:00:00+08:00	21	炽合金	5319992	6387537	0.8328706354	0.0396605064
CR-8	水晶箭行动	2024-03-07 16:00:00+08:00	21	全新装置	4646907	7975553	0.5826438618	0.0277449458
OD-7	源石尘行动・复刻	2024-03-21 16:00:00+08:00	18	轻锰矿	492773	734288	0.6710895452	0.0372827525
OD-8	源石尘行动・复刻	2024-03-21 16:00:00+08:00	18	研磨石	4392122	7608231	0.5772855740	0.0320714208
BB-6	巴别塔	2024-04-11 16:00:00+08:00	21	固源岩组	2499103	2283903	1.0942246672	0.0521059365
BB-7	巴别塔	2024-04-11 16:00:00+08:00	21	异铁组	5634620	7239761	0.7782881230	0.0370613392
BB-8	巴别塔	2024-04-11 16:00:00+08:00	21	凝胶	6160014	8773087	0.7021489699	0.0334356652
CW-8	孤星・复刻	2024-05-24 16:00:00+08:00	21	酮凝集组	1034311	1329212	0.7781384760	0.0370542131
CW-9	孤星・复刻	2024-05-24 16:00:00+08:00	21	半自然溶剂	644316	918821	0.7012421353	0.0333924826
CW-10	孤星・复刻	2024-05-24 16:00:00+08:00	21	炽合金	7159426	8598243	0.8326615100	0.0396505481
BP-5	生路	2024-06-05 16:00:00+08:00	21	晶体元件	2680908	3063233	0.8751890568	0.0416756694
BP-6	生路	2024-06-05 16:00:00+08:00	21	糖组	5771626	5939091	0.9718029241	0.0462763297
BP-7	生路	2024-06-05 16:00:00+08:00	21	扭转醇	8954720	9723583	0.9209280159	0.0438537150
HE-6	空想花庭・复刻	2024-06-25 16:00:00+08:00	21	固源岩组	1740949	1591847	1.0936660370	0.0520793351
HE-7	空想花庭・复刻	2024-06-25 16:00:00+08:00	21	凝胶	2417658	3440824	0.7026392515	0.0334590120
HE-8	空想花庭・复刻	2024-06-25 16:00:00+08:00	21	RMA70-12	2345427	4021690	0.5831943785	0.0277711609
AS-7	太阳甩在身后	2024-08-01 16:00:00+08:00	21	异铁组	5170460	6641844	0.7784675461	0.0370698831
AS-8	太阳甩在身后	2024-08-01 16:00:00+08:00	21	研磨石	5334356	7884042	0.6766016721	0.0322191272
AS-9	太阳甩在身后	2024-08-01 16:00:00+08:00	21	环烃聚质	3597547	6171119	0.5829650992	0.0277602428
SL-6	火山旅梦・复刻	2024-08-22 16:00:00+08:00	21	酮凝集组	127270	163423	0.7787765492	0.0370845976
SL-7	火山旅梦・复刻	2024-08-22 16:00:00+08:00	21	晶体元件	231512	264537	0.8751592405	0.0416742495
SL-8	火山旅梦・复刻	2024-08-22 16:00:00+08:00	21	轻锰矿	245821	316237	0.7773315583	0.0370157885
DT-6	泰拉饭	2024-09-02 16:00:00+08:00	21	聚酸酯组	1871656	1925993	0.9717875402	0.0462755972
DT-7	泰拉饭	2024-09-02 16:00:00+08:00	21	炽合金	3297704	3957604	0.8332576983	0.0396789380
DT-8	泰拉饭	2024-09-02 16:00:00+08:00	21	褐素纤维	3263346	5036667	0.6479177599	0.0308532267
CV-6	不义之财・复刻	2024-09-19 16:00:00+08:00	21	糖组	455239	468250	0.9722135611	0.0462958839
CV-7	不义之财・复刻	2024-09-19 16:00:00+08:00	21	研磨石	677328	1000203	0.6771905303	0.0322471681
CV-8	不义之财・复刻	2024-09-19 16:00:00+08:00	21	转质盐组	228570	366243	0.6240938393	0.0297187543
GO-7	追迹日落以西	2024-10-09 16:00:00+08:00	21	固源岩组	120734	110235	1.0952419830	0.0521543801
GO-8	追迹日落以西	2024-10-09 16:00:00+08:00	21	凝胶	42573	60512	0.7035464040	0.0335022097
GO-9	追迹日落以西	2024-10-09 16:00:00+08:00	21	半自然溶剂	19420	27609	0.7033938209	0.0334949439
"""


In [63]:
import pandas as pd
import numpy as np
import io


def parse_data(raw_data):
    df = pd.read_csv(io.StringIO(raw_data), sep='\t')
    df['作战开放时间'] = pd.to_datetime(df['作战开放时间'])

    return df


df = parse_data(raw_data)

# 筛选其中的固源岩组关卡
df_filtered = df[df['作战掉落物品名称'] == '固源岩组']
df_filtered


,作战名称,活动名称,作战开放时间,作战理智消耗,作战掉落物品名称,掉落数,样本数,作战掉落物品数量,单位理智掉落物品数量
12,RI-6,密林悍将归来,2020-08-25 16:00:00+08:00,15,固源岩组,5103,6586,0.774825,0.051655
25,WR-10,画中人,2021-02-05 16:00:00+08:00,18,固源岩组,11996,12881,0.931294,0.051739
34,SV-7,覆潮之下,2021-05-01 16:00:00+08:00,15,固源岩组,5446,6980,0.780229,0.052015
40,RI-6,密林悍将归来・复刻,2021-07-20 16:00:00+08:00,15,固源岩组,7450,9482,0.785699,0.052380
57,WR-10,画中人・复刻,2022-01-10 16:00:00+08:00,18,固源岩组,19802,21146,0.936442,0.052025
59,IW-7,将进酒,2022-01-25 16:00:00+08:00,21,固源岩组,31016,28216,1.099234,0.052344
70,SV-7,覆潮之下・复刻,2022-05-30 16:00:00+08:00,15,固源岩组,114679,146895,0.780687,0.052046
85,IS-8,叙拉古人,2022-11-01 16:00:00+08:00,21,固源岩组,4465516,4081599,1.094060,0.052098
95,IW-7,将进酒・复刻,2023-01-01 16:00:00+08:00,21,固源岩组,750116,685428,1.094376,0.052113
109,HE-6,空想花庭,2023-06-08 16:00:00+08:00,21,固源岩组,4888777,4468116,1.094147,0.052102


In [64]:
import numpy as np

# 掉落数 = np.array([89588, 298824, 603188, 280541, 388301, 644316, 19420])
# 样本数 = np.array([128165, 427024, 861878, 401552, 555158, 918821, 27609])
作战理智消耗 = np.array([int(x) for x in """15
18
15
15
18
21
15
21
21
21
21
21
21
21
21""".split()])
掉落数 = np.array([int(x) for x in """5103
11996
5446
7450
19802
31016
114679
4465516
750116
4888777
5411533
5385338
2499103
1740949
120734""".split()])
样本数 = np.array([int(x) for x in """6586
12881
6980
9482
21146
28216
146895
4081599
685428
4468116
4948023
4923757
2283903
1591847
110235""".split()])

# 使用假设检验判断掉落概率是否相同
from scipy import stats


def chi2_test(作站理智消耗, 掉落数, 样本数):
    p = np.sum(掉落数) / np.dot(样本数, 作站理智消耗)
    return stats.chisquare(掉落数, p * 样本数 * 作站理智消耗)


print(chi2_test(作战理智消耗, 掉落数, 样本数))


Power_divergenceResult(statistic=3.2871254317955265, pvalue=0.9984466795265587)
